<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/risk/compounded_returns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
!pip install --upgrade pandas-datareader
from pandas_datareader import DataReader as pdr

# market returns
df = pdr('F-F_Research_Data_Factors','famafrench',start=1926)[1]/100
mkt = df['Mkt-RF'] + df.RF
mkt.index.name = 'Year'
mkt.index = [int(str(x)) for x in mkt.index]
    
# example stock returns
ticker = 'CVX'
stock = pdr(ticker,'yahoo',start=1970)
stock['Year'] = [x.year for x in stock.index]
stock = stock.groupby('Year').last()['Adj Close']
stock = stock.pct_change().dropna()

# example compounding period
numyears = 10

# create dictionary of dataframes
dfs = {}
for rets, name in zip((mkt,stock),('market','stock')) :
    years = min(numyears,len(rets.index))
    compound = rets.rolling(years).apply(lambda x: (1+x).prod())

    bestyear = compound.idxmax()
    best = compound.loc[(bestyear-years+1):bestyear]
    best = (1+best).cumprod()
    best.loc[bestyear-years] = 1
    best = best.sort_index()
   
    worstyear = compound.idxmin()
    worst = rets.loc[(worstyear-numyears+1):worstyear]
    worst = (1+worst).cumprod()
    worst.loc[worstyear-years] = 1
    worst = worst.sort_index()
    
    dfs[name+'_compound'] = compound
    dfs[name+'_best'] = best
    dfs[name+'_worst'] = worst

     |████████████████████████████████| 109 kB 5.3 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


In [ ]:
# example output
dfs['market_best']

1948    1.000000e+00
1949    3.479735e+00
1950    1.556157e+01
1951    8.840907e+01
1952    4.915779e+02
1953    2.252147e+03
1954    1.224054e+04
1955    6.128750e+04
1956    3.455265e+05
1957    1.739353e+06
1958    1.172472e+07
dtype: float64